# 土地利用

In [10]:
import rasterio

# === 读取土地利用栅格 ===
landuse_path = "用地类型/Landuse_23_32650.tif"  # 替换为你本地的路径（如果不在当前目录）

with rasterio.open(landuse_path) as src:
    print("✅ 栅格信息如下：")
    print(f"CRS: {src.crs}")
    print(f"Bounds: {src.bounds}")
    print(f"Width x Height: {src.width} x {src.height}")
    print(f"Transform: {src.transform}")
    print(f"Data type: {src.dtypes[0]}")
    print(f"Number of bands: {src.count}")

✅ 栅格信息如下：
CRS: EPSG:32650
Bounds: BoundingBox(left=164849.06331957708, bottom=2473430.2121585696, right=258624.98564641795, top=2532230.9068907504)
Width x Height: 83332 x 52252
Transform: | 1.13, 0.00, 164849.06|
| 0.00,-1.13, 2532230.91|
| 0.00, 0.00, 1.00|
Data type: uint8
Number of bands: 1


In [11]:
import numpy as np

with rasterio.open(landuse_path) as src:
    band = src.read(1)  # 读取第一波段
    unique, counts = np.unique(band[band != src.nodata], return_counts=True)
    landuse_stats = dict(zip(unique, counts))
    print("✅ 土地利用分类值分布：")
    print(landuse_stats)

✅ 土地利用分类值分布：
{1: 136584103, 2: 789015851, 4: 68351, 5: 88246474, 6: 508629199, 7: 98359034, 9: 114878730}


In [9]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

# 输入原始4326栅格
src_path = "用地类型/Landuse_23.tif"
dst_path = "用地类型/Landuse_23_32650.tif"
dst_crs = "EPSG:32650"

with rasterio.open(src_path) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds
    )
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(dst_path, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest  # 分类数据用最近邻
            )

In [12]:
import geopandas as gpd
import rasterio
from rasterio.features import geometry_window, geometry_mask
from shapely.geometry import mapping
import numpy as np
from tqdm import tqdm

# === 1. 加载数据 ===
road_net = gpd.read_file("SZ_road_bet_clo_SVI.geojson")
landuse = rasterio.open("用地类型/Landuse_23_32650.tif")  # 投影后的 landuse 栅格

# === 2. 坐标系统一（确保以米为单位）
if road_net.crs != landuse.crs:
    road_net = road_net.to_crs(landuse.crs)

# === 3. 土地利用编号 → 字段名映射 ===
landuse_classes = {
    1: 'D_transport',
    2: 'D_tree',
    4: 'D_grass',
    5: 'D_farmland',
    6: 'D_building',
    7: 'D_sparseveg',
    9: 'D_water'
}

# 初始化字段
for name in landuse_classes.values():
    road_net[name] = 0.0

# === 4. 构建缓冲区（100m）和防除0长度列 ===
road_net["buffer_geom"] = road_net.geometry.buffer(100)
road_net["len_safe"] = road_net["length"].replace(0, 1)

# === 5. 主循环：栅格窗口读取 + 掩膜提取 + 统计类别密度 ===
for i, row in tqdm(road_net.iterrows(), total=len(road_net), desc="⚡ 提取土地利用密度", unit="road"):
    try:
        geom = row["buffer_geom"]
        window = geometry_window(landuse, [mapping(geom)], pad_x=0, pad_y=0)
        data = landuse.read(1, window=window)
        mask = geometry_mask([mapping(geom)], transform=landuse.window_transform(window),
                             invert=True, out_shape=data.shape)
        masked = data[mask & (data != landuse.nodata)]

        values, counts = np.unique(masked, return_counts=True)
        for v, c in zip(values, counts):
            if v in landuse_classes:
                col = landuse_classes[v]
                road_net.at[i, col] = c / row["len_safe"]
    except Exception:
        continue

# === 6. 清理中间字段并保存 ===
road_net = road_net.drop(columns=["buffer_geom", "len_safe"])
road_net.to_crs("EPSG:32650").to_file("SZ_road_bet_clo_SVI_LU.geojson", driver="GeoJSON")
print("✅ 完成：已保存为 SZ_road_bet_clo_SVI_LU.geojson")


⚡ 提取土地利用密度: 100%|██████████████████████████████████████████████████| 146332/146332 [05:53<00:00, 413.71road/s]


✅ 完成：已保存为 SZ_road_bet_clo_SVI_LU.geojson


In [13]:
road_net

,link_id,oneway,Direction,name,class,length,lanes,free_speed,capacity,link_types,...,motorcycle,bicycle,geometry,D_transport,D_tree,D_grass,D_farmland,D_building,D_sparseveg,D_water
0,29.0,FT,1.0,嘉宾路,城市主干路,84.35005,2.0,60.0,1600.0,secondary,...,296.4000,2442.1000,"LINESTRING (202539.815 2495642.37, 202545.557 ...",18.316527,7.385888,0.0,1.422643,334.498913,36.372237,53.503229
1,330130.0,FT,1.0,嘉宾路,城市主干路,97.89807,2.0,60.0,1600.0,secondary,...,400.0000,6145.0000,"LINESTRING (202442.527 2495631.464, 202451.359...",42.237809,10.715227,0.0,1.287053,283.039288,26.987253,46.681206
2,333691.0,FT,1.0,None,城市支路,202.35069,1.0,30.0,1000.0,residential,...,168.3250,743.3000,"LINESTRING (202539.815 2495642.37, 202536.814 ...",15.067900,5.233488,0.0,1.472691,196.569629,22.535134,31.321860
3,30.0,FT,1.0,嘉宾路,城市主干路,90.16309,2.0,60.0,1600.0,secondary,...,296.4000,2442.1000,"LINESTRING (202623.506 2495652.874, 202645.168...",24.355864,1.841108,0.0,0.022182,369.086729,29.757188,7.508616
4,32712.0,1,0.0,None,城市支路,181.39623,1.0,30.0,1000.0,residential,...,306.6667,2042.5000,"LINESTRING (202621.611 2495471.496, 202623.017...",7.332016,4.311005,0.0,0.066154,256.576446,18.831703,7.227273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146327,344297.0,FT,1.0,None,自行车道,107.39451,1.0,5.0,800.0,cycleway,...,NaN,NaN,"LINESTRING (188139.947 2524159.641, 188143.301...",61.176312,99.753702,0.0,21.844692,170.558067,34.461724,0.577311
146328,344825.0,FT,1.0,None,人行道路,179.19024,1.0,5.0,800.0,footway,...,0.0000,0.0000,"LINESTRING (194402.199 2511234.928, 194390.841...",69.540618,47.117522,0.0,6.998149,163.697532,7.349731,1.339359
146329,345048.0,FT,1.0,None,人行道路,137.13006,1.0,5.0,800.0,footway,...,9.9120,43.5764,"LINESTRING (190738.748 2495177.519, 190732.31 ...",76.708200,16.415073,0.0,44.658334,184.131765,15.955656,0.612557
146330,345055.0,FT,1.0,None,人行道路,138.57457,1.0,5.0,800.0,footway,...,13.3359,91.3068,"LINESTRING (190744.483 2495113.512, 190740.252...",59.267729,13.480107,0.0,27.299381,217.774444,18.206804,0.635037


# POI 香农熵